In [11]:
import sqlite3
import requests
import json


# DBファイルを保存するファイルパス
# Google Colab
path = 'jma'

# DBファイル名
db_name = 'Dsprog2.db'
# db_name = 'dsprg2.sqlite'

# DBに接続する（DBファイルが存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)


# DBへの接続を閉じる
con.close()

In [ ]:
# DBに接続
con = sqlite3.connect(path + db_name)

# SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# 実行したいSQL
# 一旦
sql = """CREATE TABLE IF NOT EXISTS offices (
    id TEXT PRIMARY KEY,
    name TEXT,
    en_name TEXT,
    weatherCodes TEXT,
    timeSeries TEXT

)"""

# SQLを実行
cur.execute(sql)

# DBへの接続を閉じる
con.close()

In [ ]:
# 地域コードのリスト（idのリスト）
office_ids = [
    "011000", "012000", "013000", "014030", "014100", 
    "015000", "016000", "017000", "020000", "030000", 
    "040000", "050000", "060000", "070000", "080000", 
    "090000", "100000", "110000", "120000", "130000", 
    "140000", "150000", "160000", "170000", "180000", 
    "190000", "200000", "210000", "220000", "230000", 
    "240000", "250000", "260000", "270000", "280000", 
    "290000", "300000", "310000", "320000", "330000", 
    "340000", "350000", "360000", "370000", "380000", 
    "390000", "400000", "410000", "420000", "430000", 
    "440000", "450000", "460040", "460100", "471000", 
    "472000", "473000", "474000"
]

# API URLのフォーマット
base_url = "https://www.jma.go.jp/bosai/forecast/data/forecast/{}.json"

# 取得したデータを格納するリスト
all_office_data = []

for office_id in office_ids:
    url = base_url.format(office_id)
    try:
        # APIリクエスト
        response = requests.get(url, timeout=10)  # 10秒でタイムアウト
        response.raise_for_status()
        
        # JSONデータを取得
        weather_data = response.json()
        
        # 必要なデータを取得
        if weather_data:
            time_defines = weather_data[0].get("timeSeries", [])[0].get("timeDefines", [])
            weather_codes = weather_data[0].get("timeSeries", [])[0].get("areas", [])[0].get("weatherCodes", [])
            
            office_info = {
                "id": office_id,
                "timeDefines": time_defines,
                "weatherCodes": weather_codes
            }
            all_office_data.append(office_info)
            
            print(f"{office_id}: データ取得成功")
        else:
            print(f"{office_id}: データが空でした")
    
    except requests.exceptions.RequestException as e:
        print(f"{office_id}: リクエストエラーが発生しました - {e}")
    except Exception as e:
        print(f"{office_id}: 想定外のエラーが発生しました - {e}")

# 取得したデータをファイルに保存（オプション）
with open('weather_data.json', 'w', encoding='utf-8') as file:
    json.dump(all_office_data, file, ensure_ascii=False, indent=4)

